<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_7_tuning_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    447      0 --:--:-- --:--:-- --:--:--   446
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-7.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [12]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-7.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,Discoloration of skin [blotchy] on the neck
3,8,arthritis
4,8,arthritis


In [13]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-7.test.csv",header=None)
test.head()

,0,1
0,3,joints poping and cracking
1,8,Arthritis
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_cadec_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.872398,4.165179,0.223246,03:52


In [0]:
# learn.fit_one_cycle(1, 1e-2)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
# learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.save_encoder('ft_fold7_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold7_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.015611,3.773019,0.371715,01:15


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.007152,3.255053,0.444305,01:27


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.458668,2.846325,0.511890,03:35


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.139489,2.420409,0.583229,03:33
1,1.801050,2.276013,0.617021,03:48


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.756915,2.110307,0.644556,03:28
1,1.446658,1.990816,0.672090,03:31


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.453204,1.866118,0.684606,03:30
1,1.334908,1.799009,0.698373,03:35


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.194128,1.705546,0.714643,03:28
1,1.119916,1.633749,0.720901,03:44


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.043061,1.612754,0.723404,03:35
1,0.947591,1.543789,0.730914,03:29


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.897418,1.545154,0.733417,03:43
1,0.942430,1.507529,0.739675,03:40
2,0.821282,1.485207,0.747184,03:40
3,0.773520,1.464881,0.743429,03:38


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.797684,1.462588,0.744681,03:41
1,0.765673,1.486840,0.737171,03:43
2,0.677531,1.429797,0.748436,03:42
3,0.636927,1.444054,0.749687,03:34


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.646203,1.478330,0.749687,03:31
1,0.683849,1.552474,0.747184,03:44
2,0.615918,1.389107,0.760951,03:43
3,0.579247,1.416945,0.764706,03:43


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.600058,1.457907,0.752190,03:29
1,0.676960,1.435684,0.760951,03:47
2,0.615720,1.422406,0.767209,03:38
3,0.518393,1.383582,0.767209,03:52


In [0]:
# save the best model

learn.save_encoder('ask-7-tuning-cadec')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [41]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[326, 8, 8, 8, 8, 8, 8, 472, 10, 13, 536, 627, 27, 27, 542, 427, 265, 31, 584, 763, 876, 37, 37, 289, 289, 289, 700, 67, 301, 49, 49, 67, 221, 49, 49, 49, 49, 216, 49, 49, 49, 100, 100, 765, 384, 53, 53, 53, 265, 337, 1031, 60, 61, 61, 62, 194, 71, 76, 76, 76, 76, 76, 289, 214, 602, 864, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 80, 80, 147, 539, 268, 88, 268, 268, 815, 815, 92, 92, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 100, 100, 100, 100, 100, 100, 337, 100, 100, 100, 100, 100, 753, 108, 111, 111, 111, 289, 243, 432, 1031, 384, 124, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 127, 127, 127, 127, 264, 139, 279, 264, 153, 994, 162, 166, 350, 129, 614, 194, 214, 289, 182, 182, 289, 182, 758, 758, 758, 758, 236, 187, 187, 187, 189, 323, 189, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 

In [42]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

521
0.6016166281755196
